    Copyright 2024 National Technology & Engineering Solutions of Sandia, LLC (NTESS). 
    Under the terms of Contract DE-NA0003525 with NTESS, the U.S. Government retains certain rights in this software.

**Step 5 - find the best models from hyperparameter sweeps**

In [ ]:
"""Imports"""

import os

import matplotlib.pyplot as plt
import numpy as np
import wandb
from config import IMAGE_DIR, MODEL_WANDB_HOST, MODEL_WANDB_PROJECT

In [ ]:
"""Fill in the sweep IDS for the hyperparameter search associated with each unsupervised loss."""

sweep_ids = {
    "chi_squared": "nj3btgc4",
    "jsd": "bhg2t26r",
    "pnll": "a4ebetbl",
    "sse": "23cf8ebd"
}

In [ ]:
"""Login to W&B."""

wandb.login(host=MODEL_WANDB_HOST)
api = wandb.Api()

In [ ]:
"""Get summary results for each run."""

sweep_val_maes = {}
sweep_val_recon_errors = {}
sweep_ood_fnrs = {}
sweep_betas = {}
sweep_lrs = {}

best_run_inds = {}
best_run_maes = []
best_run_configs = {}

for unsup_loss in sweep_ids.keys():
    sweep = api.sweep(f"{MODEL_WANDB_PROJECT}/{sweep_ids[unsup_loss]}")
    runs = sweep.runs
    sweep_val_maes[unsup_loss] = [each.summary.get("final_tuning_val_mae") for each in runs]
    sweep_val_recon_errors[unsup_loss] = [each.summary.get("final_tuning_val_recon_error") for each in runs]
    sweep_ood_fnrs[unsup_loss] = [each.summary.get("final_test_ood_fnr") for each in runs]
    sweep_betas[unsup_loss] = [each.config.get("model_beta") for each in runs]
    sweep_lrs[unsup_loss] = [each.config.get("model_init_lr") for each in runs]

    best_run_inds[unsup_loss] = np.argmin(sweep_val_maes[unsup_loss])
    best_run_maes.append(sweep_val_maes[unsup_loss][best_run_inds[unsup_loss]])
    best_run_configs[unsup_loss] = runs[best_run_inds[unsup_loss]].config

In [ ]:
"""Plot results from sweeps."""

plt_names = ["$\chi^2$",  "JSD", "PNLL", "SSE"]
best_marker_size = 100

fig, axes = plt.subplots(2, 2, figsize=(10,6), sharey=True, sharex=True)
for idx, ax in enumerate(axes.reshape(-1)):
    unsup_loss = list(sweep_ids.keys())[idx]
    ax.scatter(sweep_betas[unsup_loss], sweep_val_maes[unsup_loss], alpha=0.5, label="individual run")

    best_run = np.argmin(sweep_val_maes[unsup_loss])
    ax.scatter(
        sweep_betas[unsup_loss][best_run],
        sweep_val_maes[unsup_loss][best_run],
        color="red",
        alpha=1.0,
        label="best run",
        marker="x",
        s=best_marker_size
    )

    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_title(plt_names[idx])

axes[1,1].legend()
fig.supylabel("Tuning Validation MAE")
fig.supxlabel("Beta")
fig.tight_layout()
fig.savefig(os.path.join(
    IMAGE_DIR,
    "hp_sweep_mae_vs_beta.jpg"
), dpi=300)
plt.show()

fig, axes = plt.subplots(2, 2, figsize=(10,6), sharey=True, sharex=True)
for idx, ax in enumerate(axes.reshape(-1)):
    unsup_loss = list(sweep_ids.keys())[idx]
    ax.scatter(sweep_lrs[unsup_loss], sweep_val_maes[unsup_loss], alpha=0.5, label="individual run")

    best_run = np.argmin(sweep_val_maes[unsup_loss])
    ax.scatter(
        sweep_lrs[unsup_loss][best_run],
        sweep_val_maes[unsup_loss][best_run],
        color="red",
        alpha=1.0,
        label="best run",
        marker="x",
        s=best_marker_size
    )

    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_title(plt_names[idx])

axes[1,1].legend()
fig.supylabel("Tuning Validation MAE")
fig.supxlabel("Initial Learning Rate")
fig.tight_layout()
fig.savefig(os.path.join(
    IMAGE_DIR,
    "hp_sweep_lr_vs_beta.jpg"
), dpi=300)
plt.show()

fig, axes = plt.subplots(2, 2, figsize=(10,6), sharey=True, sharex=True)
for idx, ax in enumerate(axes.reshape(-1)):
    unsup_loss = list(sweep_ids.keys())[idx]
    ax.scatter(sweep_betas[unsup_loss], sweep_ood_fnrs[unsup_loss], alpha=0.5, label="model result")

    best_run = np.argmin(sweep_val_maes[unsup_loss])
    ax.scatter(
        sweep_betas[unsup_loss][best_run],
        sweep_ood_fnrs[unsup_loss][best_run],
        color="red",
        alpha=1.0,
        label="best model",
        marker="x",
        s=best_marker_size
    )

    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_title(plt_names[idx])

axes[1,1].legend()
fig.supylabel("OOD FNR")
fig.supxlabel("Beta")
fig.tight_layout()
fig.savefig(os.path.join(
    IMAGE_DIR,
    "hp_sweep_ood_fnr_vs_beta.jpg"
), dpi=300)
plt.show()


fig, axes = plt.subplots(2, 2, figsize=(10,6), sharex=True)
for idx, ax in enumerate(axes.reshape(-1)):
    unsup_loss = list(sweep_ids.keys())[idx]
    ax.scatter(sweep_betas[unsup_loss], sweep_val_recon_errors[unsup_loss], alpha=0.5, label="model result")

    best_run = np.argmin(sweep_val_maes[unsup_loss])
    ax.scatter(
        sweep_betas[unsup_loss][best_run],
        sweep_val_recon_errors[unsup_loss][best_run],
        color="red",
        alpha=1.0,
        label="best model",
        marker="x",
        s=best_marker_size
    )

    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_title(plt_names[idx])

axes[1,1].legend()
fig.supylabel("Tuning Validation Reconstruction Error")
fig.supxlabel("Beta")
fig.tight_layout()
fig.savefig(os.path.join(
    IMAGE_DIR,
    "hp_sweep_recon_error_vs_beta.jpg"
), dpi=300)
plt.show()